In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import zipfile
with zipfile.ZipFile('/kaggle/input/platesv2/plates.zip', 'r') as zip_obj:
   # Extract all the contents of zip file in current directory
   zip_obj.extractall('/kaggle/working/')
    
print('After zip extraction:')
print(os.listdir("/kaggle/working/"))

data_root = '/kaggle/working/plates/'
print(os.listdir(data_root))

import shutil 
from tqdm import tqdm

train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_root, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name)             
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))
        
!ls train

import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy
import random
from PIL import Image


from torchvision import transforms, models
from torchvision.transforms import functional
from torch.utils.data import ConcatDataset
from matplotlib.animation import FuncAnimation


['clean_dirty_yolo_best', 'platesv2', 'clean_dirty_resnet52_best']
After zip extraction:
['__MACOSX', '.virtual_documents', 'plates']
['.DS_Store', 'test', 'train']


100%|██████████| 21/21 [00:00<00:00, 4406.44it/s]


cleaned  dirty


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 690.9 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.7/863.7 kB 6.2 MB/s eta 0:00:00a 0:00:01


In [3]:
from ultralytics import YOLO
yolo_model = YOLO('/kaggle/input/clean_dirty_yolo_best/pytorch/default/1/clean_dirty_yolo_best.pt')
directory_cleaned = os.fsencode('/kaggle/working/val/cleaned/')
directory_dirty = os.fsencode('/kaggle/working/val/dirty/')
dir_path_cleaned, dir_path_dirty = '/kaggle/working/val/cleaned/', '/kaggle/working/val/dirty/'


In [4]:
def yolo_predict(yolo_model,dir_path_cleaned, dir_path_dirty,num_of_augment,test_time = False):
    list_of_results = []
    for each_dir in [dir_path_cleaned, dir_path_dirty]:
        for file in os.listdir(each_dir):
            filename = os.fsdecode(file)
            if (file.endswith(".jpg")):                             
                if test_time:
                    list_of_images = []
                    for i in range(num_of_augment):
                        pred_transforms = transforms.Compose([
                            transforms.RandomResizedCrop(224),
                            transforms.RandomHorizontalFlip(),
                            transforms.ColorJitter(brightness=(0.5,1.5), contrast=(1), saturation=(0.5,1.5), hue=(-0.1,0.1))         
                            ])

                        img = Image.open(each_dir+filename)
                        img = pred_transforms(img)
                        list_of_images.append(img)
                    res = yolo_model.predict(list_of_images)
                    list_of_tensors = []
                    for each_elem in res:
                        list_of_tensors.append(each_elem.probs.data)

                    inference_result = torch.stack(list_of_tensors)
                    inference_result = torch.mean(inference_result,dim=0)
                    list_of_results.append(inference_result)
                else:
                    res = yolo_model.predict(each_dir+filename)
                    list_of_results.append(res[0].probs.data)
                    
                      
            #res = torch.stack(x.float() for x in sequence)
            
        #train_dataset = torchvision.datasets.ImageFolder(dir_path_temp,train_transforms)
        #list_of_train_datasets.append(train_dataset)
    return list_of_results     
            #
            
            #print(torch.argmax(res[0].probs.data))
    

In [47]:
results = yolo_predict(yolo_model,dir_path_cleaned, dir_path_dirty, num_of_augment=30,test_time = True )
loss = torch.nn.CrossEntropyLoss()
target = torch.tensor([[1.,0.],[1.,0.],[1.,0.],[1.,0.],[0.,1.],[0.,1.],[0.,1.],[0.,1.]])
results = torch.stack(results)
loss_val = loss(results.cpu(),target)
loss_val


0: 640x640 dirty 0.52, cleaned 0.48, 40.2ms
1: 640x640 cleaned 0.85, dirty 0.15, 40.2ms
2: 640x640 cleaned 0.58, dirty 0.42, 40.2ms
3: 640x640 cleaned 0.80, dirty 0.20, 40.2ms
4: 640x640 cleaned 0.81, dirty 0.19, 40.2ms
5: 640x640 cleaned 0.60, dirty 0.40, 40.2ms
6: 640x640 cleaned 0.70, dirty 0.30, 40.2ms
7: 640x640 dirty 0.55, cleaned 0.45, 40.2ms
8: 640x640 cleaned 0.86, dirty 0.14, 40.2ms
9: 640x640 cleaned 0.90, dirty 0.10, 40.2ms
10: 640x640 dirty 0.51, cleaned 0.49, 40.2ms
11: 640x640 cleaned 0.54, dirty 0.46, 40.2ms
12: 640x640 cleaned 0.89, dirty 0.11, 40.2ms
13: 640x640 cleaned 0.64, dirty 0.36, 40.2ms
14: 640x640 cleaned 0.51, dirty 0.49, 40.2ms
15: 640x640 dirty 0.68, cleaned 0.32, 40.2ms
16: 640x640 dirty 0.60, cleaned 0.40, 40.2ms
17: 640x640 cleaned 0.66, dirty 0.34, 40.2ms
18: 640x640 cleaned 0.64, dirty 0.36, 40.2ms
19: 640x640 cleaned 0.84, dirty 0.16, 40.2ms
20: 640x640 dirty 0.67, cleaned 0.33, 40.2ms
21: 640x640 cleaned 0.83, dirty 0.17, 40.2ms
22: 640x640 cleaned

tensor(0.4466)

In [48]:
loss_val

tensor(0.4466)

In [39]:
results2 = yolo_predict(yolo_model,dir_path_cleaned, dir_path_dirty, num_of_augment=5)
results2 = torch.stack(results2)
target = torch.tensor([[1.,0.],[1.,0.],[1.,0.],[1.,0.],[0.,1.],[0.,1.],[0.,1.],[0.,1.]])
loss2 = torch.nn.CrossEntropyLoss()
loss_val2 = loss(results2.cpu(),target)



image 1/1 /kaggle/working/val/cleaned/0006.jpg: 640x640 cleaned 0.70, dirty 0.30, 61.2ms
Speed: 12.6ms preprocess, 61.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/val/cleaned/0009.jpg: 640x640 cleaned 0.97, dirty 0.03, 58.3ms
Speed: 11.5ms preprocess, 58.3ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/val/cleaned/0010.jpg: 640x640 cleaned 0.98, dirty 0.02, 55.7ms
Speed: 11.9ms preprocess, 55.7ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/val/cleaned/0019.jpg: 640x640 cleaned 0.96, dirty 0.04, 54.7ms
Speed: 12.3ms preprocess, 54.7ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/val/dirty/0006.jpg: 640x640 dirty 0.80, cleaned 0.20, 40.1ms
Speed: 12.5ms preprocess, 40.1ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /kaggle/working/val/dirty/0009.jpg: 640x640 dirty 0

In [40]:
loss_val2

tensor(0.4478)

In [9]:
resnet_model = models.resnet152()
resnet_model.fc = torch.nn.Linear(resnet_model.fc.in_features, 2)
resnet_model.load_state_dict(torch.load('/kaggle/input/clean_dirty_resnet52_best/pytorch/default/1/clean_dirty_resnet52best.pt'))

<All keys matched successfully>

In [6]:
test_dir = 'test'
shutil.copytree(os.path.join(data_root, 'test'), os.path.join(test_dir, 'unknown'))

class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path
    
val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
test_dataset = ImageFolderWithPaths('/kaggle/working/test', val_transforms)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, shuffle=False, num_workers=0)


In [15]:
test_predictions = []
test_img_paths = []
for inputs, labels, paths in tqdm(test_dataloader):
    inputs = inputs
    labels = labels
    #with torch.set_grad_enabled(False):
    preds = yolo_model(inputs)
    print(preds[0].probs.data)
    test_predictions.append(
        preds[0].probs.data[1].cpu().numpy())
    test_img_paths.extend(paths)
    
test_predictions = np.concatenate(test_predictions)

  0%|          | 0/744 [00:00<?, ?it/s]


WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.640000104904175. Dividing input by 255.
0: 224x224 cleaned 0.67, dirty 0.33, 15.2ms
Speed: 2.0ms preprocess, 15.2ms inference, 0.6ms postprocess per image at shape (1, 3, 224, 224)
tensor([0.6703, 0.3297], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.640000104904175. Dividing input by 255.
0: 224x224 cleaned 0.61, dirty 0.39, 14.1ms
Speed: 0.3ms preprocess, 14.1ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)
tensor([0.6145, 0.3855], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.273987293243408. Dividing input by 255.
0: 224x224 cleaned 0.99, dirty 0.01, 14.1ms
Speed: 0.3ms preprocess, 14.1ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)
tensor([0.9879, 0.0121], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.37856245040

  1%|          | 4/744 [00:00<00:19, 38.09it/s]

tensor([0.4921, 0.5079], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.640000104904175. Dividing input by 255.
0: 224x224 cleaned 0.77, dirty 0.23, 14.0ms
Speed: 0.4ms preprocess, 14.0ms inference, 0.4ms postprocess per image at shape (1, 3, 224, 224)
tensor([0.7671, 0.2329], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.2914161682128906. Dividing input by 255.
0: 224x224 dirty 0.62, cleaned 0.38, 14.1ms
Speed: 0.3ms preprocess, 14.1ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)
tensor([0.3829, 0.6171], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.3437039852142334. Dividing input by 255.
0: 224x224 cleaned 0.88, dirty 0.12, 14.1ms
Speed: 0.3ms preprocess, 14.1ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)
tensor([0.8801, 0.1199], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be norma

  1%|          | 8/744 [00:00<00:18, 38.93it/s]

...

 99%|█████████▉| 739/744 [00:14<00:00, 51.98it/s]

tensor([0.8928, 0.1072], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.640000104904175. Dividing input by 255.
0: 224x224 cleaned 0.69, dirty 0.31, 8.7ms
Speed: 0.3ms preprocess, 8.7ms inference, 0.5ms postprocess per image at shape (1, 3, 224, 224)
tensor([0.6912, 0.3088], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 2.640000104904175. Dividing input by 255.
0: 224x224 cleaned 0.63, dirty 0.37, 8.8ms
Speed: 0.3ms preprocess, 8.8ms inference, 0.4ms postprocess per image at shape (1, 3, 224, 224)
tensor([0.6280, 0.3720], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 1.7351655960083008. Dividing input by 255.
0: 224x224 cleaned 0.99, dirty 0.01, 7.8ms
Speed: 0.2ms preprocess, 7.8ms inference, 0.4ms postprocess per image at shape (1, 3, 224, 224)
tensor([0.9852, 0.0148], device='cuda:0')

WARNING ⚠️ torch.Tensor inputs should be normalized 0

100%|██████████| 744/744 [00:14<00:00, 49.75it/s]

tensor([0.8501, 0.1499], device='cuda:0')


ValueError: zero-dimensional arrays cannot be concatenated

In [17]:
submission_df = pd.DataFrame.from_dict({'id': test_img_paths, 'label': test_predictions})
submission_df['label'] = submission_df['label'].map(lambda pred: 'dirty' if pred > 0.5 else 'cleaned')
submission_df['id'] = submission_df['id'].str.replace('/kaggle/working/test/unknown/', '')
submission_df['id'] = submission_df['id'].str.replace('.jpg', '')
submission_df.set_index('id', inplace=True)
submission_df.to_csv("submission1.csv")
submission_df.head(n=6)

,label
id,
0000,cleaned
0001,cleaned
0002,cleaned
0003,dirty
0004,cleaned
0005,dirty
